# Imports

In [1]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.1 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from pymorphy3 import MorphAnalyzer
import pymorphy3
import numpy as np

import spacy


In [5]:
df = pd.read_excel("/kaggle/input/copy-pishu-tebe-april/_ _.xlsx")
df

,Unnamed: 0,Лицевая сторона - ссылка,Оборотная сторона - ссылка,Язык текста открытки,Текст открытки,Перевод текста открытки (если на иностранном языке),Вид орфографии,"Дата написания текста (если автор указал явно, не штемпели)",Адрес отправителя,Имя отправителя,...,Населенный пункт (откуда),Страна (куда),Населенный пункт (куда),Тег_1,Тег_2,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7
0,50,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,"{Милый Жоржик! Простите, голубчик, за молчание...",NaN,"Дореволюционная (ъ,ѣ и т.д.)",[отсутствует],г. Одесса,[отсутствует],...,"Херсонская губерния, Одесский уезд, г. Одесса",Российская Империя,"Тифлисская губерния, Тифлисский уезд, г. Тифлис",Шлю привет,Житейское,Дурные вести,Учеба,"Простите, что долго не писал",NaN,NaN
1,51,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,Добрейший Паша! Будь добр вышли мне на консерв...,NaN,"Дореволюционная (ъ,ѣ и т.д.)",xx.xx.xxxx,Москва,М.Каневски,...,"Московская губерния, Московский уезд, г. Москва",Российская Империя,"Область Войска Донского, Ростовский уезд, г. Р...",Просьба,Рассказ об искусстве,Шлю привет,NaN,NaN,NaN,NaN
2,52,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,Милая моя Олечка! Открытку твою получила. Спас...,NaN,"Дореволюционная (ъ,ѣ и т.д.)",1910-12-07 00:00:00,Одесса,[отсутствует],...,"Херсонская губерния, Одесский уезд, г. Одесса",Российская Империя,"Тифлисская губерния, Тифлисский уезд, г. Тифлис",Новости,Житейское,Дурные вести,Письма от семьи,Письма от мамы,Шлю привет,NaN
3,67,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,немецкий,Odessa. 30. 7. 01. Herzliche Grüße von [?M.Cho...,Сердечный привет от [?М. Холоденко],Латиница/дореволюционная,1901-07-30 00:00:00,Одесса,[?M.Cholodenko] [Перевод] [?М. Холоденко],...,г. Одесса,Германия,"Саксония, г. Дрезден",Шлю привет,Иностранный язык,NaN,NaN,NaN,NaN,NaN
4,68,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,иностранный язык,[отсутствует],NaN,Латиница,[отсутствует],[отсутствует],[отсутствует],...,[отсутствует],[отсутствует],[отсутствует],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46726,362229,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,NaN,Современная,[отсутствует],[отсутствует],Алеся,...,[отсутствует],[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,NaN,NaN,NaN,NaN
46727,362230,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,"С новым годом! Милая, и дорогая, Тома. От всей...",NaN,Современная,[отсутствует],[отсутствует],К.Ф. Хомченко,...,"РСФСР, г. Москва",СССР,"РСФСР, Московская обл., г. Лыткарино",Новый год,Поздравление в тексте,Поздравление на картинке,Шлю привет,NaN,NaN,NaN
46728,362231,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,Дорогие мой [^мои] всех вас я поздравляю с нов...,NaN,Современная,[отсутствует],107082 Москва Б-82 Б. почтовая ул дом 18/20 ко...,Клава,...,"РСФСР, г. Москва",СССР,"РСФСР, Московская обл., г. Лыткарино",Новый год,Поздравление на картинке,Поздравление в тексте,NaN,NaN,NaN,NaN
46729,362232,https://storage.yandexcloud.net/postcards-resi...,https://storage.yandexcloud.net/postcards-resi...,русский,Тома! Сестренка ты моя Дорогая!!! От всего сер...,NaN,Современная,[отсутствует],[отсутствует],Валера,...,[отсутствует],[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Поцелуи из писем,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46731 entries, 0 to 46730
Data columns (total 34 columns):
 #   Column                                                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                                                    --------------  ----- 
 0   Unnamed: 0                                                                                                                                                46731 non-null  int64 
 1   Лицевая сторона - ссылка                                                                                                                                  46731 non-null  object
 2   Оборотная сторона - ссылка                                                                                                                                4673

In [6]:
df_filtered = df[df["Язык текста открытки"] == "русский"][["Текст открытки",
                                                           "Дата открытки (нормализованная)",
                                                           "Доп. сведения о получателе (например, титул, \"дядя\", \"товарищ\")",
                                                           "Доп. сведения об отправителе (например, титул, \"твой дядя\", \"князь\")",
                                                           "Тег_1",
                                                           "Тег_2",
                                                           "Тег_3",
                                                           "Тег_4",
                                                           "Тег_5",
                                                           "Тег_6",
                                                           "Тег_7"]]


In [ ]:
df_filtered

,Текст открытки,Дата открытки (нормализованная),"Доп. сведения о получателе (например, титул, ""дядя"", ""товарищ"")","Доп. сведения об отправителе (например, титул, ""твой дядя"", ""князь"")",Тег_1,Тег_2,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7
0,"{Милый Жоржик! Простите, голубчик, за молчание...",1906-02-18 00:00:00,Его высокоблагородие,[отсутствует],Шлю привет,Житейское,Дурные вести,Учеба,"Простите, что долго не писал",NaN,NaN
1,Добрейший Паша! Будь добр вышли мне на консерв...,1910-05-06 00:00:00,Его высокоблагородие,[отсутствует],Просьба,Рассказ об искусстве,Шлю привет,NaN,NaN,NaN,NaN
2,Милая моя Олечка! Открытку твою получила. Спас...,1910-12-07 00:00:00,Ее высокоблагородие,мама и бабушка,Новости,Житейское,Дурные вести,Письма от семьи,Письма от мамы,Шлю привет,NaN
5,Ст. Челябинск \n18/IV \nХристос Воскресе! От д...,1910-04-18 00:00:00,Ее высокоблагородие,[отсутствует],Житейское,Дурные вести,Поздравление в тексте,NaN,NaN,NaN,NaN
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,1906-12-23 00:00:00,Господин,[отсутствует],Поздравление в тексте,NaN,NaN,NaN,NaN,NaN,Рождество
...,...,...,...,...,...,...,...,...,...,...,...
46726,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,1984-01-24 00:00:00,[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,NaN,NaN,NaN,NaN
46727,"С новым годом! Милая, и дорогая, Тома. От всей...",1988-12-24 00:00:00,[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Шлю привет,NaN,NaN,NaN
46728,Дорогие мой [^мои] всех вас я поздравляю с нов...,1981-12-29 00:00:00,[отсутствует],[отсутствует],Новый год,Поздравление на картинке,Поздравление в тексте,NaN,NaN,NaN,NaN
46729,Тома! Сестренка ты моя Дорогая!!! От всего сер...,1985-01-01 00:00:00,Сестренка,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Поцелуи из писем,NaN,NaN,NaN


In [7]:
df_filtered = df_filtered.dropna(subset=["Дата открытки (нормализованная)"])


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40058 entries, 0 to 46730
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count  Dtype 
---  ------                                                                --------------  ----- 
 0   Текст открытки                                                        40058 non-null  object
 1   Дата открытки (нормализованная)                                       40058 non-null  object
 2   Доп. сведения о получателе (например, титул, "дядя", "товарищ")       40058 non-null  object
 3   Доп. сведения об отправителе (например, титул, "твой дядя", "князь")  40058 non-null  object
 4   Тег_1                                                                 39780 non-null  object
 5   Тег_2                                                                 38057 non-null  object
 6   Тег_3                                                                 32212 non-null  object
 7   Тег_4    

In [8]:
df_filtered = df_filtered.copy()

df_filtered["Год открытки"] = pd.to_datetime(df_filtered["Дата открытки (нормализованная)"], errors='coerce').dt.year

df_filtered = df_filtered.dropna(subset=["Год открытки"])

df_filtered["Год открытки"] = df_filtered["Год открытки"].astype(int)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40056 entries, 0 to 46730
Data columns (total 12 columns):
 #   Column                                                                Non-Null Count  Dtype 
---  ------                                                                --------------  ----- 
 0   Текст открытки                                                        40056 non-null  object
 1   Дата открытки (нормализованная)                                       40056 non-null  object
 2   Доп. сведения о получателе (например, титул, "дядя", "товарищ")       40056 non-null  object
 3   Доп. сведения об отправителе (например, титул, "твой дядя", "князь")  40056 non-null  object
 4   Тег_1                                                                 39779 non-null  object
 5   Тег_2                                                                 38056 non-null  object
 6   Тег_3                                                                 32211 non-null  object
 7   Тег_4    

In [9]:
df_filtered = df_filtered[df_filtered["Год открытки"] >= 1900]
df_filtered = df_filtered.drop('Дата открытки (нормализованная)', axis=1)
df_filtered

,Текст открытки,"Доп. сведения о получателе (например, титул, ""дядя"", ""товарищ"")","Доп. сведения об отправителе (например, титул, ""твой дядя"", ""князь"")",Тег_1,Тег_2,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Год открытки
0,"{Милый Жоржик! Простите, голубчик, за молчание...",Его высокоблагородие,[отсутствует],Шлю привет,Житейское,Дурные вести,Учеба,"Простите, что долго не писал",NaN,NaN,1906
1,Добрейший Паша! Будь добр вышли мне на консерв...,Его высокоблагородие,[отсутствует],Просьба,Рассказ об искусстве,Шлю привет,NaN,NaN,NaN,NaN,1910
2,Милая моя Олечка! Открытку твою получила. Спас...,Ее высокоблагородие,мама и бабушка,Новости,Житейское,Дурные вести,Письма от семьи,Письма от мамы,Шлю привет,NaN,1910
5,Ст. Челябинск \n18/IV \nХристос Воскресе! От д...,Ее высокоблагородие,[отсутствует],Житейское,Дурные вести,Поздравление в тексте,NaN,NaN,NaN,NaN,1910
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,Господин,[отсутствует],Поздравление в тексте,NaN,NaN,NaN,NaN,NaN,Рождество,1906
...,...,...,...,...,...,...,...,...,...,...,...
46726,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,NaN,NaN,NaN,NaN,1984
46727,"С новым годом! Милая, и дорогая, Тома. От всей...",[отсутствует],[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Шлю привет,NaN,NaN,NaN,1988
46728,Дорогие мой [^мои] всех вас я поздравляю с нов...,[отсутствует],[отсутствует],Новый год,Поздравление на картинке,Поздравление в тексте,NaN,NaN,NaN,NaN,1981
46729,Тома! Сестренка ты моя Дорогая!!! От всего сер...,Сестренка,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Поцелуи из писем,NaN,NaN,NaN,1985


In [10]:
df_filtered["decade"] = (df_filtered["Год открытки"] // 10) * 10
decade_counts = df_filtered["decade"].value_counts().sort_index() 
decade_counts


decade
1900    4188
1910    9904
1920     857
1930     824
1940     964
1950    2197
1960    6087
1970    6704
1980    6323
1990    1073
2000     268
2010     229
2020     379
Name: count, dtype: int64

# Register analysis

In [11]:
df_filtered_copy = df_filtered.copy()

In [12]:
df_filtered_copy['Доп. сведения о получателе (например, титул, "дядя", "товарищ")'] = df_filtered_copy['Доп. сведения о получателе (например, титул, "дядя", "товарищ")'] \
    .str.lower() \
    .str.replace(r'[^а-яё\s]', '', regex=True)  # Удаляем всё, кроме русских букв и пробелов

df_filtered_copy

,Текст открытки,"Доп. сведения о получателе (например, титул, ""дядя"", ""товарищ"")","Доп. сведения об отправителе (например, титул, ""твой дядя"", ""князь"")",Тег_1,Тег_2,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Год открытки,decade
0,"{Милый Жоржик! Простите, голубчик, за молчание...",его высокоблагородие,[отсутствует],Шлю привет,Житейское,Дурные вести,Учеба,"Простите, что долго не писал",NaN,NaN,1906,1900
1,Добрейший Паша! Будь добр вышли мне на консерв...,его высокоблагородие,[отсутствует],Просьба,Рассказ об искусстве,Шлю привет,NaN,NaN,NaN,NaN,1910,1910
2,Милая моя Олечка! Открытку твою получила. Спас...,ее высокоблагородие,мама и бабушка,Новости,Житейское,Дурные вести,Письма от семьи,Письма от мамы,Шлю привет,NaN,1910,1910
5,Ст. Челябинск \n18/IV \nХристос Воскресе! От д...,ее высокоблагородие,[отсутствует],Житейское,Дурные вести,Поздравление в тексте,NaN,NaN,NaN,NaN,1910,1910
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,господин,[отсутствует],Поздравление в тексте,NaN,NaN,NaN,NaN,NaN,Рождество,1906,1900
...,...,...,...,...,...,...,...,...,...,...,...,...
46726,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,отсутствует,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,NaN,NaN,NaN,NaN,1984,1980
46727,"С новым годом! Милая, и дорогая, Тома. От всей...",отсутствует,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Шлю привет,NaN,NaN,NaN,1988,1980
46728,Дорогие мой [^мои] всех вас я поздравляю с нов...,отсутствует,[отсутствует],Новый год,Поздравление на картинке,Поздравление в тексте,NaN,NaN,NaN,NaN,1981,1980
46729,Тома! Сестренка ты моя Дорогая!!! От всего сер...,сестренка,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Поцелуи из писем,NaN,NaN,NaN,1985,1980


In [16]:
def count_sentences(text):
    """Подсчитывает количество предложений в тексте."""
    if isinstance(text, str):
        sentences = text.split('.')
        sentences += text.split('?')
        sentences += text.split('!')
        return len([s.strip() for s in sentences if s.strip()])


# Применяем функцию к столбцу и сохраняем результаты
df_filtered_copy['Количество предложений'] = df_filtered_copy['Текст открытки'].apply(count_sentences)

# Вычисляем описательную статистику
descriptive_stats = df_filtered_copy['Количество предложений'].describe()
descriptive_stats




count    39996.000000
mean         8.314456
std          4.097859
min          3.000000
25%          6.000000
50%          7.000000
75%         10.000000
max         61.000000
Name: Количество предложений, dtype: float64

In [25]:
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 29.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
def get_most_common_adjectives_spacy(df, text_column='Текст открытки', num_adjectives=10, language_model="ru_core_news_md"):
    """
    Извлекает самые распространенные прилагательные из столбца DataFrame с использованием spaCy.
    """
    nlp = spacy.load(language_model)  # Загрузка языковой модели
    stop_words = nlp.Defaults.stop_words # множество стоп-слов

    def process_text(text):
      if isinstance(text, str):
        doc = nlp(text)
        adjectives = [token.lemma_.lower() for token in doc if token.pos_ == "ADJ" and token.lemma_.lower() not in stop_words]
        return Counter(adjectives).most_common(num_adjectives)
      else:
        return []

    return df[text_column].apply(process_text)



most_common = get_most_common_adjectives_spacy(df_filtered_copy)
most_common


0        [(милый, 2), (уважительный, 1), (добрый, 1), (...
1                                         [(искренний, 1)]
2        [(милый, 1), (добрый, 1), (бедный, 1), (круглы...
5        [(тяжёлый, 2), (iv, 1), (весёлый, 1), (новый, ...
6        [(дорогой, 1), (христов, 1), (добрый, 1), (жив...
                               ...                        
46726    [(дорогой, 1), (новый, 1), (лююимым, 1), (двою...
46727    [(новый, 1), (милый, 1), (дорогой, 1), (хороши...
46728    [(дорогой, 1), (новый, 1), (1982, 1), (крепкий...
46729    [(дорогой, 1), (родный, 1), (1988, 1), (хороши...
46730    [(новый, 3), (добрый, 1), (больший, 1), (мален...
Name: Текст открытки, Length: 39997, dtype: object


In [ ]:
morph = MorphAnalyzer()

def lemmatize_text(text):
    words = text.split()
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmas)

df_filtered_copy['Доп. сведения о получателе (например, титул, "дядя", "товарищ")'] = df_filtered_copy['Доп. сведения о получателе (например, титул, "дядя", "товарищ")'].apply(lemmatize_text)
df_filtered_copy

,Текст открытки,"Доп. сведения о получателе (например, титул, ""дядя"", ""товарищ"")","Доп. сведения об отправителе (например, титул, ""твой дядя"", ""князь"")",Тег_1,Тег_2,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Год открытки,decade
0,"{Милый Жоржик! Простите, голубчик, за молчание...",он высокоблагородие,[отсутствует],Шлю привет,Житейское,Дурные вести,Учеба,"Простите, что долго не писал",NaN,NaN,1906,1900
1,Добрейший Паша! Будь добр вышли мне на консерв...,он высокоблагородие,[отсутствует],Просьба,Рассказ об искусстве,Шлю привет,NaN,NaN,NaN,NaN,1910,1910
2,Милая моя Олечка! Открытку твою получила. Спас...,она высокоблагородие,мама и бабушка,Новости,Житейское,Дурные вести,Письма от семьи,Письма от мамы,Шлю привет,NaN,1910,1910
5,Ст. Челябинск \n18/IV \nХристос Воскресе! От д...,она высокоблагородие,[отсутствует],Житейское,Дурные вести,Поздравление в тексте,NaN,NaN,NaN,NaN,1910,1910
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,господин,[отсутствует],Поздравление в тексте,NaN,NaN,NaN,NaN,NaN,Рождество,1906,1900
...,...,...,...,...,...,...,...,...,...,...,...,...
46726,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,отсутствовать,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,NaN,NaN,NaN,NaN,1984,1980
46727,"С новым годом! Милая, и дорогая, Тома. От всей...",отсутствовать,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Шлю привет,NaN,NaN,NaN,1988,1980
46728,Дорогие мой [^мои] всех вас я поздравляю с нов...,отсутствовать,[отсутствует],Новый год,Поздравление на картинке,Поздравление в тексте,NaN,NaN,NaN,NaN,1981,1980
46729,Тома! Сестренка ты моя Дорогая!!! От всего сер...,сестрёнка,[отсутствует],Новый год,Поздравление в тексте,Поздравление на картинке,Поцелуи из писем,NaN,NaN,NaN,1985,1980


In [ ]:
# 1. Добавляем колонку с декадами (без удаления записей)
df_filtered_copy["decade"] = (df_filtered_copy["Год открытки"] // 10) * 10

# 2. Получаем уникальные декады
decades = sorted(df_filtered_copy["decade"].unique())

# 4. Анализ по декадам
for i, decade in enumerate(decades):
    # Фильтруем записи декады (без удаления)
    decade_data = df_filtered_copy[df_filtered_copy["decade"] == decade]

    # Считаем частоту фраз, исключая "отсутствовать" 
    phrases = [
        phrase
        for phrase in decade_data['Доп. сведения о получателе (например, титул, "дядя", "товарищ")']
        if phrase != "отсутствовать"
        if phrase.strip()
    ]
    phrase_counts = Counter(phrases)

    # Сортируем результаты
    counts_df = pd.DataFrame(phrase_counts.items(), columns=['Фраза', 'Частота'])
    counts_df = counts_df.sort_values('Частота', ascending=False)

    # Вывод с новой строки и ограничением 50 записей
    print(f"\nDecade: {decade}-{decade+9}\n{'='*30}")
    print(counts_df.head(50).to_string())  
    print("="*30)


Decade: 1900-1909
                                Фраза  Частота
13               она высокоблагородие      501
0                 он высокоблагородие      283
2                             госпожа      163
26                    она высокородие      155
1                            господин      129
9                 её высокоблагородие       85
42                     он высокородие       63
32                                евб       55
40                               мама       39
58             она превосходительство       37
72       он высокоблагородие господин       36
28                        мадемуазель       31
37                     её высокородие       27
15       она высокоблагородие госпожа       23
151                  высокоблагородие       21
30                               папа       20
52                               брат       20
24                               тётя       19
16                              мадам       18
54   она высокоблагородие мадемуазель    

In [ ]:
# 1. Добавляем колонку с декадами (без удаления записей)
df_filtered_copy["decade"] = (df_filtered_copy["Год открытки"] // 10) * 10

# 2. Получаем уникальные декады
decades = sorted(df_filtered_copy["decade"].unique())

# 4. Анализ по декадам
for i, decade in enumerate(decades):
    # Фильтруем записи декады (без удаления)
    decade_data = df_filtered_copy[df_filtered_copy["decade"] == decade]


    phrases = [
        str(phrase).strip()
        for col in ['Тег_1', 'Тег_2', 'Тег_3', 'Тег_4', 'Тег_5', 'Тег_6', 'Тег_7']  
        for phrase in decade_data[col]
        if not pd.isna(phrase) and str(phrase).strip() != ""
    ]
    phrase_counts = Counter(phrases)

    # Сортируем результаты
    counts_df = pd.DataFrame(phrase_counts.items(), columns=['Фраза', 'Частота'])
    counts_df = counts_df.sort_values('Частота', ascending=False)

    print(f"\nDecade: {decade}-{decade+9}\n{'='*30}")
    print(counts_df.head(50).to_string())
    print("="*30)


Decade: 1900-1909
                               Фраза  Частота
7                          Житейское     1764
1              Поздравление в тексте     1424
0                         Шлю привет     1085
2                            Новости      974
8                   Поцелуи из писем      968
17                           Поездка      782
51                    Из путешествия      406
47                             Пасха      363
44                           Именины      353
46                         Новый год      337
11                           Просьба      337
13                      Письма семье      332
4                    Письма от семьи      305
5                             В пути      289
59                      Виды городов      283
6             Благодарность в тексте      280
21               Загадочная открытка      224
10          Поздравление на картинке      224
3                              Адрес      217
28                  Дети на картинке      182
16             

# Linguistic features extraction

In [ ]:
df_ling = df_filtered[["Текст открытки", "Год открытки"]]

In [ ]:
df_ling["Текст открытки"] = df_ling["Текст открытки"].str.replace(r'\n|\t|\r', ' ', regex=True)

df_ling["Текст открытки"] = df_ling["Текст открытки"].str.replace(r'[|]', '', regex=True)

df_ling


<ipython-input-44-b361ce0a25d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ling["Текст открытки"] = df_ling["Текст открытки"].str.replace(r'\n|\t|\r', ' ', regex=True)


,Текст открытки,Год открытки
0,"{Милый Жоржик! Простите, голубчик, за молчание...",1906
1,Добрейший Паша! Будь добр вышли мне на консерв...,1910
2,Милая моя Олечка! Открытку твою получила. Спас...,1910
5,Ст. Челябинск 18/IV Христос Воскресе! От душ...,1910
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,1906
...,...,...
46726,Дорогие Лыткаринцы! Поздравляем с Новым годом!...,1984
46727,"С новым годом! Милая, и дорогая, Тома. От всей...",1988
46728,Дорогие мой [^мои] всех вас я поздравляю с нов...,1981
46729,Тома! Сестренка ты моя Дорогая!!! От всего сер...,1985


In [ ]:
# Создаем колонку с декадами
df_ling['decade'] = (df_ling['Год открытки'] // 10) * 10

# Словарь для хранения DataFrame
decade_dfs = {}

# Группировка и сохранение
for decade, group in df_ling.groupby('decade'):
    decade_dfs[decade] = group[["Текст открытки", "Год открытки"]].copy()

    print(f"\nДекада {decade}-{decade+9} ({len(decade_dfs[decade])} записей)")


Декада 1900-1909 (4188 записей)

Декада 1910-1919 (9904 записей)

Декада 1920-1929 (857 записей)

Декада 1930-1939 (824 записей)

Декада 1940-1949 (964 записей)

Декада 1950-1959 (2197 записей)

Декада 1960-1969 (6087 записей)

Декада 1970-1979 (6704 записей)

Декада 1980-1989 (6323 записей)

Декада 1990-1999 (1073 записей)

Декада 2000-2009 (268 записей)

Декада 2010-2019 (229 записей)

Декада 2020-2029 (379 записей)


<ipython-input-45-6511342fe568>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ling['decade'] = (df_ling['Год открытки'] // 10) * 10


In [ ]:
decade_dfs[1900]

,Текст открытки,Год открытки
0,"{Милый Жоржик! Простите, голубчик, за молчание...",1906
6,Дорогие [? Папашенька] Саша Марфуша и Ваня! По...,1906
10,А.М.,1909
15,Христос Воскресе! Дорогой добрейший Ксенофонт ...,1907
21,Шлю привет и лучшие пожелания. Сейчас сижу в В...,1909
...,...,...
46700,26/V 08 Тепленько стало. В вагоне сплошь стоит...,1908
46707,[отсутствует],1907
46710,{<u>№: 16</u>} Какая интересная открытка! Выгр...,1909
46713,Глубокоуважаемый Андрей Яковлевич! По[...] оби...,1909


In [ ]:
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install stanza

In [ ]:
!nvcc --version
!pip install cupy-cuda12x

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
import stanza
import spacy
import cupy

# Инициализация моделей
if cupy.is_available():
    print("GPU доступен")
    spacy.prefer_gpu()
nlp_spacy = spacy.load("ru_core_news_lg")
stanza.download('ru')
nlp_stanza = stanza.Pipeline('ru', use_gpu=True)

GPU доступен


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
indef_list = {'некто', 'нечто', 'некоторый', 'несколько', 'некий', 'кое-кто', "кое-что", "кое-какой", "кое-чей", "кто-то", "что-то", "какой-то", "чей-то", "кто-нибудь", "что-нибудь", "какой-нибудь", "чей-нибудь", "кто-либо", "что-либо", "какой-либо", "чей-либо"}

place_adv_list = {'вблизи',
                  'вверху',
                  'вдалеке',
                  'вдали',
                  'взаперти',
                  'вне',
                  'внизу',
                  'внутри',
                  'вовне',
                  'возле',
                  'вокруг',
                  'впереди',
                  'всюду',
                  'высоко',
                  'где',
                  'далеко',
                  'далёко',
                  'изнутри',
                  'навстречу',
                  'наособицу',
                  'невдалеке',
                  'недалеко',
                  'недалечко',
                  'неподалёку',
                  'низом',
                  'одаль',
                  'одесную',
                  'около',
                  'окрест',
                  'откуда',
                  'отсюда',
                  'передом',
                  'поблизости',
                  'повсюду',
                  'поодаль',
                  'посередине',
                  'посерёдке',
                  'посреди',
                  'посредине',
                  'прочь',
                  'рядом',
                  'сверху',
                  'сзади',
                  'слева',
                  'снаружи',
                  'снизу',
                  'спереди',
                  'справа',
                  'там',
                  'вверх',
                  'вниз',
                  'доселе',
                  'досель',
                  'дотуда',
                  'дотудова',
                  'изовсюду',
                  'кое-куда',
                  'кое-откуда',
                  'кой-куда',
                  'куда',
                  'куда угодно',
                  'куда-либо',
                  'куда-нибудь',
                  'куда-то',
                  'никуда',
                  'нигде',
                  'отколь',
                  'откуда',
                  'откуда-либо',
                  'откуда-то',
                  'откудова',
                  'отовсюду',
                  'отселе',
                  'отсель',
                  'отсюда',
                  'отсюдова',
                  'отсюду',
                  'оттелева',
                  'оттель',
                  'оттоль',
                  'оттуда',
                  'оттудова',
                  'сюда',
                  'туда',
                  'тут',
                  'туда-обратно',
                  'туда-сюда'
                  }

time_adv_list = {
'анадысь',
'ввек',
'вдалеке',
'вдали',
'весной',
'весною',
'вечером',
'вечор',
'вовремя',
'впоследствии',
'впредь',
'встарь',
'вчера',
'вчерась',
'давеча',
'давно',
'дальше',
'днём',
'днесь',
'днями',
'доднесь',
'долго',
'доле',
'долее',
'доныне',
'досветла',
'дотемна',
'древле',
'ежедневно',
'ежеквартально',
'ежемесячно',
'еженочно',
'еженощно',
'заблаговременно',
'завременно',
'завтра',
'задолго',
'зараз',
'заранее',
'засветло',
'засим',
'затем',
'затемно',
'зимой',
'зимою',
'издревле',
'иногда',
'испокон',
'каждодневно',
'каждомесячно',
'когда-либо',
'летом',
'навсегда',
'надолго',
'надысь',
'накануне',
'намедни',
'насовсем',
'наутро',
'невовремя',
'недавно',
'незадолго',
'несвоевременно',
'нонеча',
'нонче',
'ночию',
'ночью',
'ныне',
'нынече',
'нынче',
'однажды',
'отныне'
'первоначально',
'поднесь',
'подоле',
'подчас',
'позднее',
'поздно',
'позже',
'пока',
'покамест',
'покуда',
'поначалу',
'поныне',
'порой',
'после'
'послезавтра',
'прежде',
'ранее',
'рано',
'раньше',
'редко',
'сегодня',
'сейгод',
'сейчас',
'скоро',
'смальства',
'смолоду',
'сперва',
'спокон',
'сразу',
'стемна',
'сыздетства',
'сызмала',
'сызмалу',
'сызмальства',
'теперича',
'теперь',
'третёвось',
'третьёвось',
'утром',
'часто',
'ща',
'щас'
}

In [ ]:
# Загрузка данных словаря абстрактных слов
abstraction_df_noun = pd.read_csv('/content/Slovar.r.ya..s.indeksom.konkretnosti.slov.csv')
abstraction_df_adj = pd.read_csv ('/content/Slovar.r.ya..s.indeksom.konkretnosti.slov_прил.csv')

In [ ]:
abstraction_df_noun

,word,score,Индекс С/A
0,туфельки,35.0000,4.555348
1,шаровары,35.0000,4.555348
2,сапожки,32.0000,4.465736
3,горсовета,31.0000,4.433987
4,плащи,31.0000,4.433987
...,...,...,...
63946,явление,0.0029,-4.843045
63947,утверждение,0.0029,-4.843045
63948,необходимость,0.0028,-4.878136
63949,признание,0.0028,-4.878136


In [ ]:
abstraction_df_adj

,w,score,Индекс С/A
0,новенький,76.0000,5.330733
1,кожаная,60.0000,5.094345
2,самодельный,49.0000,4.891820
3,шерстяной,49.0000,4.891820
4,тоненькая,46.0000,4.828641
...,...,...,...
23875,необыкновенную,0.0089,-3.721704
23876,социальную,0.0084,-3.779524
23877,радостное,0.0074,-3.906275
23878,исключительную,0.0072,-3.933674


In [ ]:
from collections import defaultdict

def normalize_noun(word):
    """Нормализация существительных с обработкой исключений"""
    try:
        parsed = morph.parse(str(word))[0]
        if 'NOUN' in parsed.tag:
            return parsed.normal_form.lower().strip()
        return str(word).lower().strip()
    except:
        return str(word).lower().strip()

def normalize_adj(word):
    """Нормализация прилагательных с обработкой исключений"""
    try:
        parsed = morph.parse(str(word))[0]
        if 'ADJF' in parsed.tag:
            return parsed.normal_form.lower().strip()
        return str(word).lower().strip()
    except:
        return str(word).lower().strip()

# Обработка существительных
normalized_dict_noun = defaultdict(list)

for raw_word, score in zip(abstraction_df_noun['word'], abstraction_df_noun['Индекс С/A']):
    normalized = normalize_noun(raw_word)
    normalized_dict_noun[normalized].append(float(score))  # Конвертация в float

abstraction_dict_noun = {}
for lemma, scores in normalized_dict_noun.items():
    abstraction_dict_noun[lemma] = sum(scores) / len(scores)  # Ручной расчет среднего

print({k: v for k, v in list(abstraction_dict_noun.items())[:5]})

# Обработка прилагательных
abstraction_df_adj = abstraction_df_adj.copy()
abstraction_df_adj['Индекс С/A'] = abstraction_df_adj['Индекс С/A'].astype(float)
abstraction_df_adj['normalized'] = abstraction_df_adj['w'].apply(normalize_adj)

abstraction_dict_adj = (
    abstraction_df_adj
    .groupby('normalized')['Индекс С/A']
    .mean()
    .to_dict()
)

print({k: float(v) for k, v in list(abstraction_dict_adj.items())[:5]})


{'туфелька': 2.5299385246000003, 'шаровары': 3.2013158180000003, 'сапожок': 2.40869740075, 'горсовет': 2.716993602, 'плащ': 2.1715183975000003}
{'абиссинская': 1.693147181, 'абиссинский': 0.306852819, 'абонементный': 1.0, 'абордажные': 1.0, 'абразивная': 1.223143551}


In [ ]:
import re
# Предварительная настройка
dim_patterns = [re.compile(rf'({s})[а-я]*$') for s in [
    'ик', 'ек', 'к', 'ок', 'ёк', 'ец', 'иц',
    'очк', 'ечк', 'оньк', 'еньк', 'ышк', 'ишк', 'ушк', 'юшк'
]]

In [ ]:
from collections import deque
class RussianTextAnalyzer:
    def __init__(self, text):
        self.text = text
        self.text_len = len(text)
        self.spacy_doc = nlp_spacy(text)
        self.stanza_doc = nlp_stanza(text)
        self.words = [token.text for token in self.spacy_doc if not token.is_punct]
        self.abstraction_dict_noun = {k: float(v) for k, v in abstraction_dict_noun.items()}
        self.abstraction_dict_adj = {k: float(v) for k, v in abstraction_dict_adj.items()}

        # Инициализируем счетчики
        self.features = {}

    def analyze(self):
        self._lexical_features()
        self._syntactic_features()
        self._morphological_features()
        return self.features

    def _lexical_features(self):
        # Лексические признаки
        pos_counts = Counter(token.pos_ for token in self.spacy_doc)

        gram_spacy = {}
        for token in self.spacy_doc:
          if token.pos_ not in gram_spacy:
            gram_spacy[token.pos_] = Counter(token.morph.to_dict().values())
          else:
            gram_spacy[token.pos_] += Counter(token.morph.to_dict().values())

        gram_stanza = {}
        for sent in self.stanza_doc.sentences:
          for word in sent.words:
            if word.upos not in gram_stanza:
              gram_stanza[word.upos] = Counter(dict(item.split('=') for item in word.feats.split('|')).values() if word.feats else {})
            else:
              gram_stanza[word.upos] += Counter(dict(item.split('=') for item in word.feats.split('|')).values() if word.feats else {})



        def analyze_abstr():
            upos_lemmas = [
                (word.upos, word.lemma.lower().strip())
                for sent in self.stanza_doc.sentences
                for word in sent.words
            ]

            scores_noun = []
            scores_adj = []
            total_nouns = 0
            total_adj = 0

            for upos, lemma in upos_lemmas:
                if upos == 'NOUN':
                    total_nouns += 1
                    if lemma in self.abstraction_dict_noun:
                        scores_noun.append(float(self.abstraction_dict_noun[lemma]))  # Явное преобразование
                if upos == 'ADJ':
                    total_adj += 1
                    if lemma in self.abstraction_dict_adj:
                        scores_adj.append(float(self.abstraction_dict_adj[lemma]))

            # Расчет статистики
            mean_noun = sum(scores_noun)/len(scores_noun) if scores_noun else None
            mean_adj = sum(scores_adj)/len(scores_adj) if scores_adj else None
            sorted_scores_noun = sorted(scores_noun)
            sorted_scores_adj = sorted(scores_adj)
            n_noun = len(sorted_scores_noun)
            n_adj = len(sorted_scores_adj)
            median_noun = (
                sorted_scores_noun[n_noun//2]
                if n_noun % 2 else
                (sorted_scores_noun[n_noun//2-1] + sorted_scores_noun[n_noun//2])/2
            ) if scores_noun else None
            median_adj = (
                sorted_scores_adj[n_adj//2]
                if n_adj % 2 else
                (sorted_scores_adj[n_adj//2-1] + sorted_scores_adj[n_adj//2])/2
            ) if scores_adj else None

            return {
                'mean_score_noun': mean_noun,
                'mean_score_adj': mean_adj,
                'median_score_noun': median_noun,
                'median_score_adj': median_adj,
                'coverage_noun': len(scores_noun)/total_nouns if total_nouns else 0.0,
                'coverage_adj': len(scores_adj)/total_adj if total_adj else 0.0,
                'total_nouns': total_nouns,
                'total_adj': total_adj
            }

        abstr_pos = analyze_abstr()
        self.features.update({
            'first_person_pronouns_sing': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=First' in t.morph
                                                and 'Number=Sing' in t.morph
                                              ),
            'first_person_pronouns_plur': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=First' in t.morph
                                                and 'Number=Plur' in t.morph
                                              ),
            'second_person_pronouns_sing': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Second' in t.morph
                                                and 'Number=Sing' in t.morph
                                              ),
            'second_person_pronouns_plur': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Second' in t.morph
                                                and 'Number=Plur' in t.morph
                                              ),
            'third_person_pronouns_masc': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Third' in t.morph
                                                and 'Gender=Masc' in t.morph
                                              ),
            'third_person_pronouns_fem': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Third' in t.morph
                                                and 'Gender=Fem' in t.morph
                                            ),
            'third_person_pronouns_neut': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Third' in t.morph
                                                and 'Gender=Neut' in t.morph
                                             ),
            'third_person_pronouns_plur': sum(
                                                1
                                                for t in self.spacy_doc
                                                if t.pos_ == 'PRON'
                                                and 'Person=Third' in t.morph
                                                and 'Number=Plur' in t.morph
                                             ),
            'demonstrative_pronouns': gram_stanza.get('DET', Counter()).get('Dem',0),
            'prepositions': pos_counts.get('ADP', 0),
            'conjunctions': pos_counts.get('CCONJ', 0) + pos_counts.get('SCONJ', 0),
            'indefinite_pronouns': sum(
                                        1
                                        for sent in self.stanza_doc.sentences
                                        for word in sent.words
                                        if word.lemma.lower() in indef_list
                                        and word.upos in {'DET', 'PRON'}
                                    ),
            'place_adverbials': sum(
                                        1
                                        for sent in self.stanza_doc.sentences
                                        for word in sent.words
                                        if word.lemma.lower() in place_adv_list
                                        and word.upos in {'ADV'}
                                    ),
            'time_adverbials': sum(
                                        1
                                        for sent in self.stanza_doc.sentences
                                        for word in sent.words
                                        if word.lemma.lower() in time_adv_list
                                        and word.upos in {'ADV'}
                                    ),
            'noun_anim': sum(
                                  1
                                  for t in self.spacy_doc
                                  if t.pos_ == 'NOUN'
                                  and 'Animacy=Anim' in t.morph
                                ),
            'noun_inan': sum(
                              1
                              for t in self.spacy_doc
                              if t.pos_ == 'NOUN'
                              and 'Animacy=Inan' in t.morph
                            ),
            'noun_abstr_index': abstr_pos['mean_score_noun'],
            'adj_abstr_index': abstr_pos['mean_score_adj'],
            'latin_letters': sum(1
                                    for token in self.spacy_doc
                                    if any('LATN' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'propr_name': sum(1
                                    for token in self.spacy_doc
                                    if any('Name' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'patr_name': sum(1
                                    for token in self.spacy_doc
                                    if any('Patr' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'sur_name': sum(1
                                    for token in self.spacy_doc
                                    if any('Surn' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'praedicative': sum(1
                                    for token in self.spacy_doc
                                    if any('PRED' in parse.tag for parse in morph.parse(token.text)) or any('Prdx' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'geo_name': sum(
                              1
                              for token in self.spacy_doc
                              if token.pos_ == "PROPN"
                              if any('Geox' in parse.tag for parse in morph.parse(token.text))
                              ),
            'intj': sum(1
                                    for token in self.spacy_doc
                                    if any('INTJ' in parse.tag for parse in morph.parse(token.text))
                                  )
        })


    def _syntactic_features(self):
        # Синтаксические признаки
        sentence_lengths = [len(sent.text.split()) for sent in self.spacy_doc.sents]

        def is_minor_constituent(word, sentence):
            """Определяет тип сочинительной конструкции"""
            if word.head == 0:
                return None

            head = sentence.words[word.head - 1]

            # Определяем тип конструкции
            if word.deprel == 'conj':
                if head.upos == 'NOUN' and word.upos == 'NOUN':
                    return 'NOUN'
                elif head.upos == 'ADJ' and word.upos == 'ADJ':
                    return 'ADJ'
                elif head.upos == 'VERB' and word.upos == 'VERB':
                    return 'VERB'
                elif head.upos == 'ADV' and word.upos == 'ADV':
                    return 'ADV'
                elif head.deprel in {'obj', 'nsubj', 'nmod'}:
                    return 'NOUN'  # Для именных дополнений
                elif head.deprel == 'amod':
                    return 'ADJ'   # Для определений
                elif head.deprel == 'advmod':
                    return 'ADV'   # Для обстоятельств

            return None

        def extract_minor_coordinations(text):
            doc = self.stanza_doc
            coordination_counts = {
                'NOUN': 0,
                'ADJ': 0,
                'VERB': 0,
                'ADV': 0,
                'OTHER': 0
            }

            for sentence in doc.sentences:
                for word in sentence.words:
                    if word.deprel == 'conj':
                        const_type = is_minor_constituent(word, sentence)
                        if const_type:
                            if const_type in coordination_counts:
                                coordination_counts[const_type] += 1
                            else:
                                coordination_counts['OTHER'] += 1

            return coordination_counts

        coordination_counts = extract_minor_coordinations(text)

        def analyze_tree(text):
            doc = self.stanza_doc
            features = {
                'max_tree_depth': 0,
                'avg_np_length': 0.0,
                'avg_vp_length': 0.0,
                'inversion_count': 0,
                'ellipsis_count': 0
            }

            all_depths = []
            np_lengths = []
            vp_lengths = []

            for sentence in doc.sentences:
                # 2.1. Глубина дерева
                depths = _calculate_depths(sentence)
                all_depths.extend(depths)

                # 2.2. Длина NP/VP
                nps = _extract_phrases(sentence, 'NP')
                vps = _extract_phrases(sentence, 'VP')
                np_lengths.extend([len(np) for np in nps])
                vp_lengths.extend([len(vp) for vp in vps])

                # 2.3. Инверсии
                features['inversion_count'] += _count_inversions(sentence)

                # 2.4. Эллипсис
                features['ellipsis_count'] += sum(1 for word in sentence.words
                                                if word.deprel == 'orphan')

            # Расчет итоговых значений
            if all_depths:
                features['max_tree_depth'] = max(all_depths)
            if np_lengths:
                features['avg_np_length'] = sum(np_lengths)/len(np_lengths)
            if vp_lengths:
                features['avg_vp_length'] = sum(vp_lengths)/len(vp_lengths)

            return features

        def _calculate_depths(sentence):
            """Расчет глубины дерева с использованием BFS"""
            depths = []
            root = next((word for word in sentence.words if word.head == 0), None)
            if not root:
                return []

            queue = deque([(root, 0)])
            visited = set()

            while queue:
                word, depth = queue.popleft()
                if word.id in visited:
                    continue
                visited.add(word.id)
                depths.append(depth)

                # Добавляем дочерние узлы
                children = [w for w in sentence.words if w.head == word.id]
                for child in children:
                    queue.append((child, depth + 1))

            return depths

        def _extract_phrases(sentence, phrase_type):
            """Извлечение фраз с фильтрацией"""
            phrases = []
            targets = {
                'NP': ['NOUN', 'PROPN', 'PRON'],
                'VP': ['VERB', 'AUX']
            }

            for word in sentence.words:
                if word.upos in targets[phrase_type]:
                    phrase = _get_phrase(sentence, word.id)
                    if _is_valid_phrase(phrase_type, sentence, phrase):
                        phrases.append(phrase)

            return phrases

        def _get_phrase(sentence, head_id):
            """Поиск в ширину для более точного определения границ фразы"""
            phrase = []
            queue = deque([head_id])
            visited = set()

            while queue:
                current_id = queue.popleft()
                if current_id in visited:
                    continue
                visited.add(current_id)

                phrase.append(current_id)
                current_word = sentence.words[current_id-1]

                # Добавляем только непосредственные зависимые
                children = [w.id for w in sentence.words
                          if w.head == current_id
                          and w.deprel not in ['punct', 'cc', 'mark']]
                queue.extend(children)

            return sorted(phrase)

        def _is_valid_phrase(phrase_type, sentence, phrase_ids):
            """Проверка валидности извлеченной фразы"""
            if len(phrase_ids) < 1:
                return False

            main_word = sentence.words[phrase_ids[0]-1]

            if phrase_type == 'NP':
                return main_word.upos in ['NOUN', 'PROPN', 'PRON']
            elif phrase_type == 'VP':
                return main_word.upos in ['VERB', 'AUX']
            return False

        def _count_inversions(sentence):
            """Подсчет инверсий"""
            inversions = 0
            for word in sentence.words:
                if word.deprel == 'nsubj':
                    verb = sentence.words[word.head-1]
                    # Более гибкое условие для русского языка
                    if word.id > verb.id and (word.id - verb.id) >= 1:
                        context = sentence.words[verb.id-1:word.id]
                        if not any(w.deprel == 'advmod' for w in context):
                            inversions += 1
            return inversions

        tree_analysis = analyze_tree(text)

        def count_syllables_ru(word):
            """Пподсчет слогов для русского языка"""
            vowels = 'аеёиоуыэюя'
            word = word.lower()
            count = 0
            prev_vowel = False

            for char in word:
                if char in vowels:
                    if not prev_vowel:  # Учитываем только последовательные гласные как один слог
                        count += 1
                    prev_vowel = True
                else:
                    prev_vowel = False

            # Гарантируем минимум 1 слог для коротких слов
            return max(1, count)

        def flesch_kincaid_russian(text):
            """Адаптированная формула для русского языка: 206.835 − 1.52 × ASL − 65.14 × ASW"""
            try:

                # Сбор статистики
                sentences = self.stanza_doc.sentences
                num_sentences = len(sentences)
                words = [word.text for sent in sentences for word in sent.words]
                num_words = len(words)

                if num_sentences == 0 or num_words == 0:
                    return 0.0

                # Подсчет слогов
                total_syllables = sum(count_syllables_ru(word) for word in words)

                # Расчет показателей
                ASL = num_words / num_sentences  # Average Sentence Length
                ASW = total_syllables / num_words  # Average Syllables per Word

                # Применение формулы
                score = 206.835 - 1.52 * ASL - 65.14 * ASW

                # Ограничение диапазона 0-100
                return max(0, min(100, round(score, 2)))

            except Exception as e:
                print(f"Ошибка при обработке текста: {e}")
                return 0.0

        flesch_kincaid_index = flesch_kincaid_russian(self.text)


        self.features.update({
            'mean_sentence_length': sum(sentence_lengths)/len(sentence_lengths) if sentence_lengths else 0,
            'subordinate_clauses': sum(1 for token in self.spacy_doc if token.dep_ == 'mark'),
            'type-token ratio': len(set(self.words)) / len(self.words) * 100 if self.words else 0,
            'word length': sum(len(word) for word in self.words) / len(self.words),
            'noun_coordination': coordination_counts['NOUN'],
            'adj_coordination': coordination_counts['ADJ'],
            'verb_coordination': coordination_counts['VERB'],
            'adv_coordination': coordination_counts['ADV'],
            'other_coordination': coordination_counts['OTHER'],
            'max_tree_depth': tree_analysis['max_tree_depth'],
            'avg_np_length': tree_analysis['avg_np_length'],
            'avg_vp_length': tree_analysis['avg_vp_length'],
            'inversion_count': tree_analysis['inversion_count'],
            'ellipsis_count': tree_analysis['ellipsis_count'],
            'flesch_kincaid_index': flesch_kincaid_index
        })

    def _morphological_features(self):
        # Морфологические признаки

        self.features.update({
            'perfect_aspect': sum(1 for token in self.spacy_doc if  token.pos_ == 'VERB' and 'Aspect=Perf' in token.morph),
            'imperfect_aspect': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Aspect=Imp' in token.morph),
            'past_tense': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Tense=Past' in token.morph),
            'present_tense': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Tense=Pres' in token.morph),
            'fut_tense': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Tense=Fut' in token.morph),
            'ind_mood_verb': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Mood=Ind' in token.morph),
            'imp_mood_verb': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Mood=Imp' in token.morph),
            'cnd_mood_verb': sum(1 for token in self.spacy_doc if token.pos_ == 'AUX' and 'Mood=Cnd' in token.morph),
            'gerunds': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'VerbForm=Conv' in token.morph),
            'participles': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'VerbForm=Part' in token.morph),
            'infinitives': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'VerbForm=Inf' in token.morph),
            'finite_verbs': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'VerbForm=Fin' in token.morph),
            'passive_voice': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Voice=Pass' in token.morph),
            'active_voice': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Voice=Act' in token.morph),
            'middle_voice': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and 'Voice=Mid' in token.morph),
            'neg_polarity': sum(1 for token in self.spacy_doc if token.pos_ == 'PART' and 'Polarity=Neg' in token.morph),
            'first_pers_verb_sing': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Sing' and 'Person=First') in token.morph),
            'second_pers_verb_sing': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Sing' and 'Person=Second') in token.morph),
            'third_pers_verb_sing': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Sing' and 'Person=Third') in token.morph),
            'first_pers_verb_plur': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Plur' and 'Person=First') in token.morph),
            'second_pers_verb_plur': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Plur' and 'Person=Second') in token.morph),
            'third_pers_verb_plur': sum(1 for token in self.spacy_doc if token.pos_ == 'VERB' and ('Number=Plur' and 'Person=Third') in token.morph),
            'trans_verb': sum(
                              1
                              for token in self.spacy_doc
                              if token.pos_ == "VERB"
                              if any('tran' in parse.tag for parse in morph.parse(token.text))
                              ),
            'intr_verb': sum(
                              1
                              for token in self.spacy_doc
                              if token.pos_ == "VERB"
                              if any('intr' in parse.tag for parse in morph.parse(token.text))
                              ),
            'not_inv_verb': sum(
                              1
                              for token in self.spacy_doc
                              if token.pos_ == "VERB"
                              if any('excl' in parse.tag for parse in morph.parse(token.text))
                              ),
            'sing_noun': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Number=Sing' in token.morph),
            'plur_noun': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Number=Plur' in token.morph),
            'plr_tantum_noun': sum(
                                    1
                                    for token in self.spacy_doc
                                    if token.pos_ == "NOUN"
                                    if any('Pltm' in parse.tag for parse in morph.parse(token.text))
                                   ),
            'sing_tantum_noun': sum(
                                    1
                                    for token in self.spacy_doc
                                    if token.pos_ == "NOUN"
                                    if any('Sgtm' in parse.tag for parse in morph.parse(token.text))
                                   ),
            'noun_fem': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Gender=Fem' in token.morph),
            'noun_masc': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Gender=Masc' in token.morph),
            'noun_neut': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Gender=Neut' in token.morph),
            'noun_case_nom': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Nom' in token.morph),
            'noun_case_gen': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Gen' in token.morph),
            'noun_case_dat': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Dat' in token.morph),
            'noun_case_acc': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Acc' in token.morph),
            'noun_case_loc': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Loc' in token.morph),
            'noun_case_ins': sum(1 for token in self.spacy_doc if token.pos_ == 'NOUN' and 'Case=Ins' in token.morph),
            'noun_case_voc': sum(
                                    1
                                    for token in self.spacy_doc
                                    if token.pos_ == "NOUN"
                                    if any('voct' in parse.tag for parse in morph.parse(token.text))
                                ),
            'fixed_noun': sum(
                                    1
                                    for token in self.spacy_doc
                                    if token.pos_ == "NOUN"
                                    if any('Fixd' in parse.tag for parse in morph.parse(token.text))
                                ),
            'sing_adj': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Number=Sing' in token.morph),
            'plur_adj': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Number=Plur' in token.morph),
            'adj_fem': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Gender=Fem' in token.morph),
            'adj_masc': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Gender=Masc' in token.morph),
            'adj_neut': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Gender=Neut' in token.morph),
            'adj_case_nom': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Nom' in token.morph),
            'adj_case_gen': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Gen' in token.morph),
            'adj_case_dat': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Dat' in token.morph),
            'adj_case_acc': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Acc' in token.morph),
            'adj_case_loc': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Loc' in token.morph),
            'adj_case_ins': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Case=Ins' in token.morph),
            'adj_case_voc': sum(
                                    1
                                    for token in self.spacy_doc
                                    if token.pos_ == "ADJ"
                                    if any('voct' in parse.tag for parse in morph.parse(token.text))
                                ),
            'adj_degree_pos': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Degree=Pos' in token.morph),
            'adj_degree_comp': sum(1 for token in self.spacy_doc if token.pos_ == 'ADJ' and 'Degree=Cmp' in token.morph),
            'adj_degree_super': sum(1
                                    for token in self.spacy_doc
                                    if token.pos_ == 'ADJ'
                                    if any('Supr' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'full_adj': sum(1 for token in self.spacy_doc if any('ADJF' in parse.tag for parse in morph.parse(token.text))),
            'shrot_adj': sum(1 for token in self.spacy_doc if any('ADJS' in parse.tag for parse in morph.parse(token.text)) and 'StyleVariant=Short' in token.morph),
            'adv_degree_pos': sum(1 for token in self.spacy_doc if token.pos_ == 'ADV' and 'Degree=Pos' in token.morph),
            'adv_degree_comp': sum(1 for token in self.spacy_doc if token.pos_ == 'ADV' and 'Degree=Cmp' in token.morph),
            'quant_num': sum(1 for token in self.spacy_doc if token.pos_ == 'NUM'),
            'anum_num': sum(1
                                    for token in self.spacy_doc
                                    if token.pos_ == 'ADJ'
                                    if any('Anum' in parse.tag for parse in morph.parse(token.text))
                                  ),
            'dim_nouns': sum(1 for t in self.spacy_doc  if t.pos_ == 'NOUN' and any(p.search(t.lemma_.lower()) for p in dim_patterns)),
            'dim_adj': sum(1 for t in self.spacy_doc  if t.pos_ == 'ADJ' and any(p.search(t.lemma_.lower()) for p in dim_patterns))
        })


In [ ]:
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Путь к папке на Google Drive, куда будем сохранять файл 
output_path = '/content/drive/My Drive/'  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.path.exists(output_path)

True

In [ ]:
total_rows = len(df_ling) # всего строк
processed_rows = 0 # обработано строк
features_list = []

# Обрабатываем тексты и сохраняем результаты
for idx, text in enumerate(df_ling['Текст открытки']):
    if pd.isna(text) or len(text.strip()) == 0:
        processed_rows += 1
        features_list.append({})
        continue

    try:
        analyzer = RussianTextAnalyzer(text)
        features = analyzer.analyze()
        features_list.append(features)
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        features_list.append({})  # Добавляем пустой словарь при ошибке

    processed_rows += 1

    # Выводим статистику (каждые 100 обработанных строк)
    if processed_rows % 100 == 0:
        percentage_complete = (processed_rows / total_rows) * 100
        print(f"Обработано строк: {processed_rows}/{total_rows} ({percentage_complete:.2f}%)")


# Создаем временный DataFrame с результатами
temp_df = pd.DataFrame(features_list).add_suffix('_abs')

# Удаляем существующие столбцы перед объединением
existing_cols = df_ling.columns.intersection(temp_df.columns)
df_ling = df_ling.drop(columns=existing_cols)

# Объединяем с исходным DataFrame
df_ling = pd.concat([df_ling.reset_index(drop=True),
                    temp_df.reset_index(drop=True)], axis=1)

print(f"Всего обработано строк: {processed_rows}")

# Сохраняем DataFrame в CSV файл на Google Drive
file_name = 'processed_data.csv' 
df_ling.to_csv(output_path + file_name, index=False) 
df_ling.to_csv('output_with_params.csv', index=False, encoding='utf-8')

print(f"DataFrame сохранен в файл: {output_path + file_name}")


Обработано строк: 100/39997 (0.25%)
Обработано строк: 200/39997 (0.50%)
Обработано строк: 300/39997 (0.75%)
Обработано строк: 400/39997 (1.00%)
Обработано строк: 500/39997 (1.25%)
Обработано строк: 600/39997 (1.50%)
Обработано строк: 700/39997 (1.75%)
Обработано строк: 800/39997 (2.00%)
Обработано строк: 900/39997 (2.25%)
Обработано строк: 1000/39997 (2.50%)
Обработано строк: 1100/39997 (2.75%)
Обработано строк: 1200/39997 (3.00%)
Обработано строк: 1300/39997 (3.25%)
Обработано строк: 1400/39997 (3.50%)
Обработано строк: 1500/39997 (3.75%)
Обработано строк: 1600/39997 (4.00%)
Обработано строк: 1700/39997 (4.25%)
Обработано строк: 1800/39997 (4.50%)
Обработано строк: 1900/39997 (4.75%)
Обработано строк: 2000/39997 (5.00%)
Обработано строк: 2100/39997 (5.25%)
Обработано строк: 2200/39997 (5.50%)
Обработано строк: 2300/39997 (5.75%)
Обработано строк: 2400/39997 (6.00%)
Обработано строк: 2500/39997 (6.25%)
Обработано строк: 2600/39997 (6.50%)
Обработано строк: 2700/39997 (6.75%)
Обработано

In [ ]:
df_ling.to_csv('output_with_params.csv', index=False, encoding='utf-8')
